# DATA ANALYSIS USING PYTHON PANDAS LIBRARY

Oct 2023 <br>
By Irina Overeem

We will be looking at data on river discharge of the Upper Colorado River. 
Tabular data like this with a combination of dates, name and data quality strings, and numbers are best handled by spreadsheets where entries such as dates and times are in some useful format. In Python the Python Data Analysis Library (a.k.a. Pandas) is really useful for this purpose.

I use one discharge data file downloaded for the USGS station at Kremmling, CO, for the Upper Colorado. 
And we will look at a file generated by a commonly used electronic setup - a campbell scientific datalogger.

In this notebook will use partially cleaned up files, because the headers in either the USGS files and to some extent also the Campbell stations is unique for each station, making it difficult to deal with it in an automated way. To prepare the file I cut of the header in a text-editor, cut the line that has 15 s ..., and saved it as a *.csv file. 

Python Learning Objectives:
1. Load csv data from a file using the Pandas library.
2. Get info on the data in a dataframe, access data in DataFrames.
3. Create plots of data in DataFrames.
4. Save figures to file.

In [ ]:
# Set up with the modules we will use

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd


## SOME REVIEW NOTES FROM LAST WEEK

### Reading file data into a numpy array: loadtxt and genfromtxt

If our goal is to read the contents of a text file into a numpy array, numpy provides functions that makes this a bit easier:

`loadtxt()` - simple and effective when you have all numbers plus some comments or header lines <br>
`genfromtxt()` - useful when there are strings embedded in the data portion of the file <br>


In [ ]:
help(np.loadtxt)

In [ ]:

#Example of loadtxt on glacial data
#Our file has 2 header lines that we need to skip; we use the skiprows parameter
#It is comma-separated; we use the delimiter parameter

har = np.loadtxt("hardangerjokulen2006-2007.csv", skiprows=2, delimiter=",")
print(har)

### What is the Pandas Module?

The Python module **Pandas** is designed to efficiently work with tabular data.

DataFrame: a two-dimensional data structure that holds data like a two-dimension array or a table with rows and columns.

### load a csv-file into a pandas dataframe

In [ ]:
# read the data file posted with this excercise into a pandas dataframe

data = pd.read_csv('USGS09058000_discharge_NoHead_20112021.csv')

# print the first five lines to see whether it populated the fields
# with the head() method, you can see the structure of the data without having to print the entire dataframe.

print(data)


We have just created a Data Frame.<br>

A DataFrame is a 2-dimensional data structure that can store data of different types (including characters, integers, floating point values, factors and more) in columns. It is similar to a spreadsheet, or an database/ SQL table.

To get quick insight in what is in a dataframe, you can use the head() method or the tail() method.

You can check the type of the object data using the type module. The type() method or the  __class__ attribute tell us what type of object the variable points to.

Then: each column in a DataFrame also has its own type. We can use data.dtypes to view the data type for each column. int64 are numeric integer values, object are strings (letters and numbers), and float64 are numbers with decimals.

In [ ]:
# to get insight in the content of a dataframe, you can use the head() method or the tail() method:
data.head()
#data.tail()

In [ ]:
# the type() function works too

print(type(data))
print(data.__class__)

In [ ]:
# 

data.dtypes

In [ ]:
# create a list with new column names as strings
new_column_names = ['Agency', 'SiteNo', 'OldDateTime', 'Discharge_cfs']

# assign the new column names to the dataframe
data.columns = new_column_names

The values in each column of a Pandas DataFrame can be accessed using the specific column name. <br>
If we want to access more than one column at once, we use a list of column names. 

In [ ]:
# for example, if we just want to select the discharge in cu ft/sec
data[['Discharge_cfs']].head()

# or as a second example, we just want to select time and discharge
data[['OldDateTime','Discharge_cfs']].head()

### Manipulating data in a dataframe

In [ ]:
# Calculations
# We can also call the data using column names and manipulate the values in the column
# For example, here we convert the discharge from cubic feet per second to metric units m3/sec 

data['Discharge_m3sec'] = data['Discharge_cfs'] * 0.028316847 
data['Discharge_m3sec'].head()

len(data['Discharge_m3sec'])

data.head()

# So we added a column with the newly calculated parameter.

#INSTEAD You can also replace the existing values in Discharge_cfs column by setting the column name equal to the output of the applied function.

When Pandas imported the data, it read the station name (a number) as an integer and removed the initial zero. We can fix the station name by replacing the values of that column with a string.
Remember that the goal is to automate the process for multiple stations. Instead of writing the corrected station name ourselves, let’s build it from the values available in the DataFrame.

The Pandas method unique returns a numpy array of the unique elements in the DataFrame. We want the first (and only) entry in that array, which has the index 0. We can build a string with the correct station name by casting that value as a string and concatenating it with an initial zero.

We can replace all values in the ‘Station’ column with this string through assignment and check the object type of each column to make sure it is no longer an integer.

In [ ]:
# manipulation of strings in one of the columns of the dataframe

new_station_name = "0" + str(data['SiteNo'].unique()[0])
data['SiteNo'] = new_station_name

data.head()

In [ ]:
# max, min and basic stats can be easily found with a couple of math methods built into pandas

print(data['Discharge_m3sec'].max())
print(data['Discharge_m3sec'].min())

print(data['Discharge_m3sec'].mean())
print(data['Discharge_m3sec'].std())

# also very handy for inquiries is to find the index of these max values
peakflowindex=data['Discharge_m3sec'].idxmax()

print(peakflowindex)

peakflowday=data['DateTime'][peakflowindex]
print(peakflowday)




## Handling Date and Time stamps

Different programming languages and software packages handle date and time stamps in their own unique ways. Pandas has a set of functions for creating and managing timeseries.

We need to convert the USGS entries in the DateTime column into a format that Pandas can work with. Luckily, the to_datetime function in the Pandas library can convert it directly. Note that this conversion takes up a bit of time, wait till you see those first five lines reported....


In [ ]:
# using the pd.to_datetime functionality

data['DateTime'] = pd.to_datetime(data['OldDateTime'])

data.head()

In [ ]:
#The entries in our DataFrame data are indexed by the number in bold on the left side of each row. 
# We can display a slice of the data using index ranges:

data[0:5]

 ## <font color = green> IN-CLASS PRACTICE </font> 

Can you print out the river discharge on 1/10/11?
Can you print out the very last entry in the dataset? When was this?

In [ ]:
# your print statements here

data["Discharge_m3sec"][9]

#last=(len(data))-1

#print(data["Discharge_m3sec"][last])
#print(data["DateTime"][last])


## Creating data subsets and removing columns

We can select specific ranges of our data in both the row and column directions using either label or integer-based indexing.

loc: indexing via labels or integers <br>
iloc: indexing via integers <br>

To select a subset of rows AND columns from our DataFrame, we can use the loc method and the integer indices for both rows and columns. 

In [ ]:
data.loc[0:2, ['DateTime', 'Discharge_m3sec']]

In [ ]:
data.iloc[0:2,-2:]

In [ ]:
print('With iloc:', data.iloc[0:2,-2:].shape)
print('With loc:', data.loc[0:2, ['DateTime', 'Discharge_m3sec']].shape)

Since we can call individual columns (or lists of columns) from a DataFrame, the simplest way to remove columns is by creating a new DataFrame with only the columns we want.

In [ ]:
discharge_data = data[['DateTime', 'Discharge_m3sec']]
discharge_data.head()

In [ ]:
# it is possible to separate components of the date time
# here we use dt.year to just select the year in a separate column
data['year'] = data['DateTime'].dt.year
data.head()


#Q2011=  data[data.year == 2011]
#print(Q2011)

 ## <font color = green> IN CLASS PRACTICE </font> 

First, use the iloc method to create a new dataframe that has DateTime and Discharge over just 2020. 
To check whether you have selected indeed the whole year, print out the last 12 values of the newly created data frame.

Then use the dt.year functionality to create a new dataframe that has DateTime and Discharge over just 2019. 

In [ ]:
# your code here using iloc


In [ ]:
# your code using the dt.year functionality


## Creating Plots of Pandas Dataframes

Pandas is well integrated with the matplotlib library that we used earlier in the tutorials. 
We can either use the same functions we used before with to plot data in NumPy arrays or we can use the plotting functions built into Pandas.

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters 
register_matplotlib_converters()

plt.plot(data['DateTime'], data['Discharge_cfs'])
plt.title('Gauging Station ' + data['SiteNo'][0])

In [ ]:
data.plot(x='DateTime', y='Discharge_cfs', title='Station ' + data['SiteNo'][0])

In [ ]:
data.plot(x='DateTime', y='Discharge_cfs', title='Station ' + data['SiteNo'][0])
plt.xlabel('Time')
plt.ylabel('Discharge (cfs)')

# to save your figure to a file, specify a name before the final plotting command
plt.savefig('USGStestplot.png')

plt.show()

 ## <font color = green> IN-CLASS PRACTICE </font> 

1. make a plot of discharge over 2019 only
2. relabel the title to call this USGS station at Kremmling, CO
3. plot the line in black
4. save your figure to a png file


In [ ]:
# your code goes here

## Using Logical Operators with DataFrames to Filter Values

Another useful data wrangling option is the ability to filter data from an existing pandas dataframe.

Filtering data is easily done using dataframe.column_name == "value". Your output will contain all rows that meet the criteria.

For example, you can filter using a comparison operator on numeric values. For example, you can select all rows from the dataframe that have discharge greater than 8500 cfs by filtering on the Discharge_in_cfs column.



In [ ]:
# create new dataframe from filter on values in the `Discharge_cfs` column greater than 8500 cfs

High_discharge_days = data[data.Discharge_cfs > 8500]

# print new dataframe
High_discharge_days

 ## <font color = green> IN-CLASS PRACTICE </font> 
 
1. what is the lowest flow recorded at the Kremmling USGS station?
2. Use the `nsmallest` method to create new dataframe from filter on values in the `Discharge_cfs` column that captures the 10 lowest flow days.


In [ ]:
# your code here

data.Discharge_cfs.min()

low_flow = data.nsmallest(1, "Discharge_cfs")
print("The lowest flow recorded,", low_flow["Discharge_cfs"].item(), "cfs, occurred on", low_flow["DateTime"].item())

data.nsmallest(10, "Discharge_cfs")
#print("The tenth lowest flow is 308.8 cfs")

ten_lowest = data[data.Discharge_cfs < 309]
print(ten_lowest)



In [ ]:
# your code goes here